In [23]:
from ANNPDE.PDE.shapes import (
    ElipseShape, 
    CircleShape, 
    LineShape
)
from ANNPDE.PDE import ReverseChauchyPDE
import plotly.graph_objs as go
from random import randint
import numpy as np

In [24]:
SEED = randint(1, 1000000)
print('Seed:', SEED)

Seed: 827801


In [25]:
F_EXPR = 'E ** x1 * sin(x2) * cos(t)'
G_EXPR = ['E ** x1 * sin(x2)', 'E ** x1 * cos(x2)']
H_EXPR = 'E ** x1 * sin(x2)'
TIME_CONDITION = (0, np.pi / 2)
Xs_symbol, t_symbol = ['x1', 'x2'], 't'

T_SAMPLE = 128
E_SAMPLE = 256
D_SAMPLE = 2048
CENTER = np.array([0, 0])
RADIUS = 10

In [26]:
time = LineShape(
    seed=SEED,
    n=T_SAMPLE,
    start_point=TIME_CONDITION[0],
    end_point=TIME_CONDITION[1],
    cross_sample_generate=1,
    even_sample=True
)
time_sample = time.get()
time.plot()

In [27]:
shape = CircleShape(
    seed=SEED,
    edge_n=E_SAMPLE,
    domain_n=D_SAMPLE,
    center=CENTER,
    radius=RADIUS,
    cross_sample_generate=1,
    even_sample=True
)

edge_sample, domain_sample = shape.get()
shape.plot()

In [28]:
pde = ReverseChauchyPDE(
    f_function=F_EXPR,
    g_function=G_EXPR,
    h_function=H_EXPR,
    x_symbols=Xs_symbol,
    domain_sample=domain_sample,
    edge_sample=edge_sample,
    time_sample=time_sample,
    time_symbol=t_symbol
)

In [29]:
print('Shape Domain:', pde.domain_sample.shape)
print('Shape Edge:', pde.edge_sample.shape)
print('Shape Time:', pde.time_sample.shape)

Shape Domain: (2048, 2)
Shape Edge: (256, 2)
Shape Time: (128,)


In [30]:
print('Shape Domain:', pde.domain_sample.shape)
print('Shape Edge:', pde.edge_sample.shape)
print('Shape Time:', pde.time_sample.shape)

Shape Domain: (2048, 2)
Shape Edge: (256, 2)
Shape Time: (128,)


In [31]:
print('Shape Domain:', pde.domain_sample.shape)
print('Shape Edge:', pde.edge_sample.shape)
print('Shape Time:', pde.time_sample.shape)

Shape Domain: (2048, 2)
Shape Edge: (256, 2)
Shape Time: (128,)


In [32]:
print('Shape Domain:', pde.domain_sample.shape)
print('Shape Edge:', pde.edge_sample.shape)
print('Shape Time:', pde.time_sample.shape)

Shape Domain: (2048, 2)
Shape Edge: (256, 2)
Shape Time: (128,)


In [33]:
print('Shape Domain:', pde.domain_sample.shape)
print('Shape Edge:', pde.edge_sample.shape)
print('Shape Time:', pde.time_sample.shape)

Shape Domain: (2048, 2)
Shape Edge: (256, 2)
Shape Time: (128,)


In [34]:
print('Shape Domain:', pde.domain_sample.shape)
print('Shape Edge:', pde.edge_sample.shape)
print('Shape Time:', pde.time_sample.shape)

Shape Domain: (2048, 2)
Shape Edge: (256, 2)
Shape Time: (128,)


In [35]:
print('Shape Domain:', pde.domain_sample.shape)
print('Shape Edge:', pde.edge_sample.shape)
print('Shape Time:', pde.time_sample.shape)

Shape Domain: (2048, 2)
Shape Edge: (256, 2)
Shape Time: (128,)
